## Binance top 200 coins by volume and momentum

In [1]:
coins = 5

### Obtengo la data usando la API de Binance

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_top_volume_coins_list_api(count=200):
    # Obtener la lista de todos los símbolos de trading
    symbols_url = "https://api.binance.com/api/v3/exchangeInfo"
    symbols_response = requests.get(symbols_url)
    symbols_data = symbols_response.json()
    
    # Filtrar solo los pares con USDT
    usdt_symbols = [symbol['symbol'] for symbol in symbols_data['symbols'] if symbol['symbol'].endswith('USDT')]
    
    # Calcular la fecha de inicio (7 días atrás)
    start_time = int((datetime.now() - timedelta(days=7)).timestamp() * 1000)
    
    # Obtener el volumen para cada símbolo
    volumes = []
    for symbol in usdt_symbols:
        klines_url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval=1d&startTime={start_time}"
        klines_response = requests.get(klines_url)
        klines_data = klines_response.json()
        
        # Calcular el volumen promedio
        total_volume = sum(float(kline[7]) for kline in klines_data) #7 es quote asset volume
        avg_volume = total_volume / len(klines_data) if klines_data else 0
        volumes.append((symbol[:-4], avg_volume))  # Removemos 'USDT' del símbolo
    
    # Ordenar por volumen descendente y tomar los primeros 'count'
    top_volumes = sorted(volumes, key=lambda x: x[1], reverse=True)[:count]
    
    return top_volumes  # Ahora retornamos la lista completa de tuplas (moneda, volumen)

def get_top_volume_coins_df_api(count=200):
    volumes = get_top_volume_coins_list_api(count)
    df = pd.DataFrame(volumes, columns=['coin', 'average volume last 7 days'])

    # Ordenar el DataFrame antes de formatear los números
    df = df.sort_values('average volume last 7 days', ascending=False)

   # Formatear los números después de ordenar
    df['average volume last 7 days'] = df['average volume last 7 days'].apply(lambda x: f'{x:.2f}')

    return df

/Users/egrados/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
api_top_coins_list = get_top_volume_coins_list_api(coins)
api_top_coins_list

[('BTC', 1174442646.6691563),
 ('ETH', 548002519.2091274),
 ('USDC', 426954156.6250858),
 ('SOL', 340129189.38739854),
 ('FDUSD', 328811259.9649857)]

In [4]:
api_top_coins_df = get_top_volume_coins_df_api(coins)
api_top_coins_df

,coin,average volume last 7 days
0,BTC,1174687015.86
1,ETH,548087276.98
2,USDC,426982398.12
3,SOL,340191424.23
4,FDUSD,328957454.74


### Obtengo la data usando el wraper de Python de Binance

In [5]:
from binance.client import Client
import pandas as pd
from datetime import datetime, timedelta

# Inicializar el cliente de Binance
# Nota: Para el uso de la API pública, no necesitamos proporcionar claves API
client = Client()

def get_top_volume_coins_list_wraper(count=200):
    # Obtener la información de todos los símbolos de trading
    exchange_info = client.get_exchange_info()
    
    # Filtrar solo los pares con USDT
    usdt_symbols = [symbol['symbol'] for symbol in exchange_info['symbols'] if symbol['symbol'].endswith('USDT')]
    
    # Calcular la fecha de inicio (7 días atrás)
    start_time = int((datetime.now() - timedelta(days=7)).timestamp() * 1000)
    
    # Obtener el volumen para cada símbolo
    volumes = []
    for symbol in usdt_symbols:
        klines = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_1DAY, startTime=start_time)
        
        # Calcular el volumen promedio
        total_volume = sum(float(kline[7]) for kline in klines) #7 es quote asset volume
        avg_volume = total_volume / len(klines) if klines else 0
        volumes.append((symbol[:-4], avg_volume))  # Removemos 'USDT' del símbolo
    
    # Ordenar por volumen descendente y tomar los primeros 'count'
    top_volumes = sorted(volumes, key=lambda x: x[1], reverse=True)[:count]
    
    return top_volumes  # Ahora retornamos la lista completa de tuplas (moneda, volumen)

def get_top_volume_coins_df_wraper(count=200):
    volumes = get_top_volume_coins_list_wraper(count)
    df = pd.DataFrame(volumes, columns=['coin', 'average volume last 7 days'])

    # Ordenar el DataFrame antes de formatear los números
    df = df.sort_values('average volume last 7 days', ascending=False)

    # Formatear los números después de ordenar
    df['average volume last 7 days'] = df['average volume last 7 days'].apply(lambda x: f'{x:.2f}')
    

    return df





In [6]:
#wraper_top_coins_list = get_top_volume_coins_list_wraper(coins)
#wraper_top_coins_list

In [7]:
#wraper_top_coins_df = get_top_volume_coins_df_wraper(coins)
#wraper_top_coins_df

### Conclusion
Mejor uso el API directamente, ya que evito que CS50 tenga que instalar un paquete adicional de python. 

Las variables que pasan a la siguiente etapa son: 


api_top_coins_df 

api_top_coins_list

Trabajare con la lista pero primero tengo que eliminar los valores de volumen, que hago a continuacion

In [8]:
top_coins = [item[0] for item in api_top_coins_list]
top_coins

['BTC', 'ETH', 'USDC', 'SOL', 'FDUSD']

## Obtener datos para cada criptomoeda

In [9]:
import pandas as pd
from binance.client import Client
from datetime import datetime, timedelta
import os
import sqlite3

client = Client()

def get_crypto_data(crypto_list, lookback, reference_low):
    # Convertir la fecha de referencia a timestamp
    reference_date = datetime.strptime(reference_low, "%d-%m-%Y")
    start_date = reference_date - timedelta(days=lookback)
    start_timestamp = int(start_date.timestamp() * 1000)

    dataframes = []

    # Crear la carpeta 'data' si no existe
    if not os.path.exists('data'):
        os.makedirs('data')

    for crypto in crypto_list:
        # Obtener los datos de Binance
        klines = client.get_historical_klines(
            symbol=f"{crypto}USDT",
            interval=Client.KLINE_INTERVAL_1DAY,
            start_str=start_timestamp
        )

        # Crear el DataFrame
        df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
        
        # Convertir el timestamp a datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        
        # Establecer el timestamp como índice
        df.set_index('timestamp', inplace=True)
        
        # Convertir las columnas necesarias a float
        for col in ['open', 'high', 'low', 'close', 'volume']:
            df[col] = df[col].astype(float)
        
        # Guardar el DataFrame como CSV
        filename = f"data/{crypto}_DF_{lookback}.csv"
        df.to_csv(filename)
        
        dataframes.append(df)
        print(f"Datos de {crypto} guardados en {filename}")

    return dataframes

def create_db(crypto_list):
    # Crear la base de datos
    conn = sqlite3.connect('crypto_db.db')
    cursor = conn.cursor()

    # Crear una tabla para cada criptomoneda
    for crypto in crypto_list:
        cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {crypto} (
            date TEXT PRIMARY KEY,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            volume REAL
        )
        ''')

    conn.commit()
    conn.close()
    print("Base de datos creada: crypto_db.db")

# Ejemplo de uso:
# crypto_list = ['BTC', 'ETH', 'ADA']
# dataframes = get_crypto_data(crypto_list, 30, "01-06-2023")
# create_db(crypto_list)

In [10]:
dataframes = get_crypto_data(top_coins, 600, "05-07-2024")

Datos de BTC guardados en data/BTC_DF_600.csv
Datos de ETH guardados en data/ETH_DF_600.csv
Datos de USDC guardados en data/USDC_DF_600.csv
Datos de SOL guardados en data/SOL_DF_600.csv
Datos de FDUSD guardados en data/FDUSD_DF_600.csv


In [11]:
dataframes[0]

,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
timestamp,,,,,,,,,,,
2022-11-14,16331.78,17190.00,15815.21,16619.46,380210.77750,1668470399999,6273920325.33682730,9215128,189502.41248000,3127495594.21076420,0
2022-11-15,16617.72,17134.69,16527.72,16900.57,282461.84391,1668556799999,4758545355.78683170,7200265,141015.70658000,2375750102.41831840,0
2022-11-16,16900.57,17015.92,16378.61,16662.76,261493.40809,1668643199999,4362538338.56532390,6297404,129644.72244000,2162757428.72151800,0
2022-11-17,16661.61,16751.00,16410.74,16692.56,228038.97873,1668729599999,3785204632.01422130,5352083,113347.39771000,1881536908.06648980,0
2022-11-18,16692.56,17011.00,16546.04,16700.45,214224.18184,1668815999999,3588467560.37312090,5114799,106366.35569000,1781845353.09400520,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-09-01,58974.00,59076.59,57201.00,57301.86,20705.15741,1725235199999,1202176847.39105850,2153180,9651.50531000,560452387.89854740,0
2024-09-02,57301.77,59425.69,57128.00,59132.13,22895.01461,1725321599999,1333092369.44656860,1966119,11295.25452000,657496577.63048030,0
2024-09-03,59132.12,59809.65,57415.00,57487.73,22828.18447,1725407999999,1335076815.85992480,2208758,10979.79204000,642252673.11321130,0


## Analisis core de las monedas mas importantes de Binance

In [ ]:
import pandas as pd
import os
from datetime import datetime

def analyze_momentum(pivot_low_date, reference_high_date):
    # Convertir las fechas de string a datetime, fechas en formato DD-MM-YYYY
    pivot_low_date = datetime.strptime(pivot_low_date, "%d-%m-%Y")
    reference_high_date = datetime.strptime(reference_high_date, "%d-%m-%Y")

    results = []

    # Leer todos los archivos CSV en la carpeta 'data'
    for filename in os.listdir('data'):
        if filename.endswith('.csv'):
            coin = filename.split('_')[0]  # Extraer el nombre de la moneda del nombre del archivo
            df = pd.read_csv(os.path.join('data', filename), index_col='timestamp', parse_dates=True)

            # Asegurarse de que las columnas necesarias son de tipo float
            for col in ['low', 'high']:
                df[col] = df[col].astype(float)

            # Encontrar el pivot_low
            pivot_low = df.loc[pivot_low_date.strftime("%Y-%m-%d"), 'low']

            # Encontrar el max_high después del pivot_low_date
            max_high = df.loc[pivot_low_date:, 'high'].max()

            # Encontrar el reference_high
            reference_high = df.loc[reference_high_date.strftime("%Y-%m-%d"), 'high']

            # Calcular el cambio porcentual
            change_pct = ((reference_high - pivot_low) / pivot_low) * 100

            results.append({
                'coin': coin,
                'pivot_low': pivot_low,
                'max_high': max_high,
                'reference_high': reference_high,
                'change_pct': change_pct
            })

    # Crear el DataFrame final
    result_df = pd.DataFrame(results)

    # Ordenar el DataFrame por change_pct de mayor a menor
    result_df = result_df.sort_values('change_pct', ascending=False)

    return result_df

# Ejemplo de uso:
aa_df = analyze_momentum(pivot_low_date="05-08-2024", reference_high_date="25-08-2024")
aa_df

In [ ]:
# segunda version con fomrato YYYY-MM-DD

import os
from datetime import datetime

def analyze_momentum(pivot_low_date, reference_high_date):
    # Las fechas ya están en formato YYYY-MM-DD, no necesitamos convertirlas

    results = []

    # Leer todos los archivos CSV en la carpeta 'data'
    for filename in os.listdir('data'):
        if filename.endswith('.csv'):
            coin = filename.split('_')[0]  # Extraer el nombre de la moneda del nombre del archivo
            df = pd.read_csv(os.path.join('data', filename), index_col='timestamp', parse_dates=True)

            # Asegurarse de que las columnas necesarias son de tipo float
            for col in ['low', 'high']:
                df[col] = df[col].astype(float)

            # Encontrar el pivot_low
            pivot_low = df.loc[pivot_low_date, 'low']

            # Encontrar el max_high después del pivot_low_date
            max_high = df.loc[pivot_low_date:, 'high'].max()

            # Encontrar el reference_high
            reference_high = df.loc[reference_high_date, 'high']

            # Calcular el cambio porcentual
            change_pct = ((reference_high - pivot_low) / pivot_low) * 100

            results.append({
                'coin': coin,
                'pivot_low': pivot_low,
                'max_high': max_high,
                'reference_high': reference_high,
                'change_pct': change_pct
            })

    # Crear el DataFrame final
    result_df = pd.DataFrame(results)

    # Ordenar el DataFrame por change_pct de mayor a menor
    result_df = result_df.sort_values('change_pct', ascending=False)

    return result_df

# Ejemplo de uso:
aa_df = analyze_momentum(pivot_low_date="2024-08-5", reference_high_date="2024-08-25")
aa_df